# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


## Описание данных

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data.duplicated().sum()

54

In [15]:
data = data.drop_duplicates()

In [16]:
data['education'] = data['education'].str.lower()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

Посмотрим, есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
print( data.groupby('children')['debt'].agg('mean'))

children
0    0.075353
1    0.092327
2    0.094542
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64


In [23]:
# общее количество просрочек по кредиту с разбивкой по количеству детей в семье
print(data.groupby('children')['debt'].sum())

children
0    1063
1     444
2     194
3      27
4       4
5       0
Name: debt, dtype: int64


**Вывод:** 

По мере увеличения количества детей вероятность просрочки увеличивается. Исключением являются семьи с 3 детьми, хотя и у них вероятность просрочки выше, чем у семей без детей. 

Семьи же с 5 детьми в нашей выборке не имели ни одной просрочки по кредиту. Но таких семей в нашей выборке всего 9 - этого количества недостаточно, чтобы делать какие-либо выводы.

Проверим, есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
print(data.groupby('family_status')['debt'].mean().sort_values())

family_status
вдовец / вдова           0.066246
в разводе                0.070648
женат / замужем          0.075575
гражданский брак         0.092861
Не женат / не замужем    0.097639
Name: debt, dtype: float64


In [25]:
print(data.groupby('family_status')['dob_years'].median().sort_values(ascending=False))

family_status
вдовец / вдова           58
в разводе                46
женат / замужем          43
гражданский брак         41
Не женат / не замужем    36
Name: dob_years, dtype: int64


**Вывод:** 

Такая зависимость есть. Но, возможно, в дейсвительности это отражение зависимости между возрастом и вероятностью вовзрата кредита в срок.


А есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
print(data.groupby('total_income_category')['debt'].mean().sort_values(ascending=False))

total_income_category
E    0.090909
C    0.084891
A    0.080000
B    0.070602
D    0.060172
Name: debt, dtype: float64


Справочно:
* Ежемесячный доход от 0  до 30 000 - Категория E
* Ежемесячный доход от 30 001  до 50 000 - Категория D
* Ежемесячный доход от 50 001  до 200 000 - Категория C
* Ежемесячный доход от 200 001  до 1 000 000 - Категория B
* Ежемесячный доход выше 1 000 000 - Категория A

**Вывод:** 

Существует зависимость между уровнем дохода и возвратом кредита в срок, но она не линейная. Заемщики из категории Е - наимение обеспеченные - ожидаемо имеют наибольший риск просрочки по кредиту. 

Однако, заемщики с доходом выше 1 000 000 имеют чуть меньшую вероятность просрочки чем заемщики с доходом 50 001 до 200 000 в месяц, что контринтуитивно. 

Посмотрим, как разные цели кредита влияют на его возврат в срок:

In [27]:
print(data.groupby('purpose_category')['debt'].mean().sort_values(ascending=False))

purpose_category
операции с автомобилем      0.093436
получение образования       0.092504
проведение свадьбы          0.078744
операции с недвижимостью    0.072531
Name: debt, dtype: float64


In [28]:
# Получаем средний возраст заемщика по различным целям кредита - проверка гипотезы на связь цели кредита и возраста.
print(data.groupby('purpose_category')['dob_years'].mean().sort_values(ascending=False))

purpose_category
операции с автомобилем      43.505957
получение образования       43.420155
проведение свадьбы          43.191910
операции с недвижимостью    43.168681
Name: dob_years, dtype: float64


**Вывод:** 

Разные цели кредита влияют на вероятность его возврат в срок. Так, наиболее рискованной целью кредита является автокредит, а наименее рискованной - операции с недвижимостью.


## Общий вывод.

&nbsp; &nbsp; В ходе исследования мы выяснили, что с увеличением количества детей уменьшается вероятность погашения кредита в срок, семейное положение влияет на погашение кредита. (При этом гипотеза о наличии связи между возрастом и вероятностью погашения кредита не подтвердилась.)

&nbsp; &nbsp; Также, вопреки ожиданиям, наиболее обеспеченная группа заемщиков не является наиболее надежной.  А обеспечение исполнения кредитных обязательств залогом - ипотекой - приводит к наименьшим просрочкам по кредитам. 

&nbsp; &nbsp; Любопытно, что примерно с такой же дисциплинированностью, что  и ипотечники, молодожены погашают кредиты, взятые на проведение свадьбы.




    